# 1. CVI reading the datasets (Creator: Juan Vasquez @ Infinite Analytics)

## Importing the libraries

In [1]:
# Importing the libraries
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import h3
from h3 import h3 as l3lib
import geopandas as gpd
from shapely.geometry import Polygon
import boto3
from geopy.distance import geodesic
from scipy.stats import spearmanr

Links suggested by Vishak

https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt
https://h3geo.org/

In what follows I will trace the cvs files coing from the respective R codes

## get_datasets.R

INPUT 
- 22-02_CVI_state_county_tract_updated.xlsx 
- CVI Indicators_ForFinal_072722.xlsx


OUTPUT
Sourcing "get_datasets.R" gives the following csv files 

- CVI_master.csv - all details as to each indicator, including source file information and notes
- Checkoutput.txt - output capture during data loading, used for diagnostics
- CVI_indicators_current.csv - most important meta-data for each indicator
- CVI_data_current.csv - indicator values (columns) for each census tract (rows)


In [2]:
# Create an ExcelFile object
excel_file = pd.ExcelFile("./22-02_CVI_state_county_tract_updated.xlsx")

# Get the sheet names
sheetnames = excel_file.sheet_names

# Print the sheet names
for sheet in sheetnames:
    print(sheet)

State
County
Tract
tl_2010_US_tracts
county_names


In [3]:
# Getting the CVI County Data
# Add the path where Data folder is stored
tractsraw = pd.read_excel("./22-02_CVI_state_county_tract_updated.xlsx", sheet_name="Tract")
tractsraw.head()

,TRACTCE10,NAMELSAD10,NAME10,FIPS,STATE,STATEFP10,County_Name,COUNTYFP10,GEOID10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,Shape_Length,Shape_Area
0,502,Census Tract 5.02,5.02,56025,WY,56,Natrona,25,56025000502,G5020,S,1950484,0,42.845903,-106.275220,0.070177,0.000215
1,1000,Census Tract 10,10.00,56025,WY,56,Natrona,25,56025001000,G5020,S,5070560,320735,42.822806,-106.392940,0.180837,0.000593
2,902,Census Tract 9.02,9.02,56025,WY,56,Natrona,25,56025000902,G5020,S,3813417,0,42.816308,-106.338814,0.108228,0.000420
3,1401,Census Tract 14.01,14.01,56025,WY,56,Natrona,25,56025001401,G5020,S,2891789616,8214702,43.208380,-106.334985,2.577982,0.321234
4,700,Census Tract 7,7.00,56025,WY,56,Natrona,25,56025000700,G5020,S,1653040,0,42.836399,-106.331264,0.061731,0.000182


In [4]:
# At this level the valus in the FIPS column coincide with the values in the FIPS column in https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt
tractsraw['FIPS'].head()

0    56025
1    56025
2    56025
3    56025
4    56025
Name: FIPS, dtype: int64

Checking the output files 

In [5]:
# Importing the data from the file
CVI_master = pd.read_csv("./CVI_master.csv")

# take a look at the dataset
CVI_master.head()

,Path To File,GEOID Column Name,Data Column Name,Subset Column Name,Indicator Name,Adverse Direction,Replace NA with median,Clean/Final Data,Converted to Tract?,Category,...,URL,Download Date,Geographic Level,Missing States?,Document Name & Dropbox Link,File Type,Notes,Verified,FracTractsNA,GeographicScale
0,"Baseline Vulnerability, Health/Health/LifeExpe...",FIPS Code,"Life expectancy at birth, as of 2010-2015.",NaN,Life Expectancy,-1.0,1,YES,NaN,Health,...,NaN,44413,Census Tract,NaN,LifeExpectancy,CSV,W Verified. This is a rate.,True,0.080882,Tract
1,"Baseline Vulnerability, Health/Health/PLACES__...",LocationName,Data_Value,Measure,Physical health not good for >=14 days among a...,1.0,1,YES,NaN,Health,...,Link,44551,Census Tract (Model based),NaN,PLACES__Local_Data_for_Better_Health__Census_T...,CSV,5/25 - NJ included in 2020 data release. 5/17:...,True,0.010198,Tract
2,"Baseline Vulnerability, Health/Health/PLACES__...",LocationName,Data_Value,Measure,Mental health not good for >=14 days among adu...,1.0,1,YES,NaN,Health,...,Link,44551,Census Tract (Model based),NaN,PLACES__Local_Data_for_Better_Health__Census_T...,CSV,5/25 - NJ included in 2020 data release. 5/17:...,True,0.010198,Tract
3,"Baseline Vulnerability, Health/Health/CTY_HEAL...",FIPS,NEW_DRUG_OD_DEATH_RATE,NaN,Fatal Drug Overdose,1.0,1,YES,NaN,Health,...,Link,44760,Census Tract,NaN,CTY_HEALTH_2021_GTL,CSV,5/25: Still many NAs (including all of NJ). W ...,True,0.000068,County
4,"Baseline Vulnerability, Health/Health/PLACES__...",LocationName,Data_Value,Measure,Binge drinking among adults aged >=18 years,1.0,1,YES,NaN,Health,...,Link,44551,Census Tract (Model based),NaN,PLACES__Local_Data_for_Better_Health__Census_T...,CSV,5/25 - NJ included in 2020 data release. 5/17:...,True,0.010198,Tract


## get_cvi_toxpi.R 

INPUT 
- CVI_indicators_current.csv  - most important meta-data for each indicator
- CVI_data_current.csv  - indicator values (columns) for each census tract (rows)

OUTPUT
Sourcing "get_cvi_toxpi.R" gives the following csv files 

- CVI-pct/CVI_data_pct.csv - indicator data transformed to percentiles
- CVI-pct/CVI-pct-allinone.csv - ToxPI treating all indicators as equally weighted (not used)
- CVI-pct/CVI-pct-cat-XXXX.csv - Category domain-specific ToxPI files for category XXXX

subcategory domains

- CVI-pct/CVI-pct-comb.csv - overall CVI ToxPI
<span style="color:red;"> 
- CVI-pct/CVI-pct-comb-baseline.csv - ToxPI for only Baseline vulnerability categories
</span>
- CVI-pct/CVI-pct-comb-climate.csv - ToxPI for only Climate change risk categories

Note - Files with ".gis.csv" are reformatted versions for use by ToxPI-gis python scripts to create ESRI GIS maps. Files with ".Rdata" are Rdata files with the ToxPI slices, model, and results stored as R objects.

In [6]:
# Importing the data from the file
CVI_indicators_current = pd.read_csv("./CVI_indicators_current.csv")

# take a look at the dataset
CVI_indicators_current.head()

,Indicator Name,Adverse Direction,Replace NA with median,Category,Subcategory,Parameters,Agency or data source,Year of data release,Geographic Level,GeographicScale
0,Life Expectancy,-1.0,1,Health,Overall Physical Health,Life Expectancy,Policy Map,2010-2015,Census Tract,Tract
1,Physical health not good for >=14 days among a...,1.0,1,Health,Overall Physical Health,Self-Reported Physical Health,"PLACES (BRFSS, RWJ, CDC)","BRFSS 2019 or 2018, Census Bureau 2010 data, a...",Census Tract (Model based),Tract
2,Mental health not good for >=14 days among adu...,1.0,1,Health,Mental Health & Deaths of Despair,Self-Reported Mental Health,"PLACES (BRFSS, RWJ, CDC)","BRFSS 2019 or 2018, Census Bureau 2010 data, a...",Census Tract (Model based),Tract
3,Fatal Drug Overdose,1.0,1,Health,Mental Health & Deaths of Despair,"Drug Overdose Deaths per 100,000 People",County Health Rankings & Roadmaps,2021,Census Tract,County
4,Binge drinking among adults aged >=18 years,1.0,1,Health,Mental Health & Deaths of Despair,Alcohol Abuse,"PLACES (BRFSS, RWJ, CDC)","BRFSS 2019 or 2018, Census Bureau 2010 data, a...",Census Tract (Model based),Tract


In [7]:
# Importing the data from the file
CVI_data_current = pd.read_csv("./CVI_data_current.csv")

# take a look at the dataset
CVI_data_current.head()

,STATE,County_Name,GEOID.State,GEOID.County,GEOID.Tract,LatLong,Life Expectancy,Self-Reported Physical Health,Self-Reported Mental Health,"Drug Overdose Deaths per 100,000 People",...,Surface PM2.5,Snowfall,Standardized Precip Index,Total Precipitation,Coastal Flooding - Annualized Frequency,Riverine Flooding - Annualized Frequency,Sea Level Rise,Hurricane - Annualized Frequency,Tornado - Annualized Frequency,Winter Weather - Annualized Frequency
0,AL,Autauga,1,1001,1001020100,"+32.4771112,-086.4903033",73.1,12.8,15.1,8.384539,...,-2.089116,-0.019121,18.664745,4.583092,NaN,1.083333,NaN,0.059841,0.003192,0.576606
1,AL,Autauga,1,1001,1001020200,"+32.4757580,-086.4724678",76.9,15.1,17.4,8.384539,...,-2.089116,-0.019121,18.664745,4.583092,NaN,1.083333,NaN,0.059841,0.001502,0.576606
2,AL,Autauga,1,1001,1001020300,"+32.4740243,-086.4597033",NaN,14.3,16.7,8.384539,...,-2.089116,-0.019121,18.664745,4.583092,NaN,1.083333,NaN,0.059841,0.002028,0.576606
3,AL,Autauga,1,1001,1001020400,"+32.4710782,-086.4446805",75.4,12.1,13.2,8.384539,...,-2.089116,-0.019121,18.664745,4.583092,NaN,1.083333,NaN,0.059841,0.002296,0.576606
4,AL,Autauga,1,1001,1001020500,"+32.4589157,-086.4218165",79.4,11.3,14.0,8.384539,...,-2.089116,-0.019121,18.664745,4.583092,NaN,1.083333,NaN,0.059841,0.003608,0.576606


In what follows comes the important part where we get the scores for each baseline vulnerability and the the the total baseline vulnerability

In [8]:
# THIS DATA GIVES DETAILED INFORMATION ABOUT THE INDICATORS FOR EACH CATEGORY
# Importing the data from the file
CVI_pct_cat_Health = pd.read_csv("/Users/juanvasquez/Documents/Work/Work_Juan/Infinite_Analytics/Projects/Data_projects/Wellness_index/CVI-main-Vasquez/CVI-pct/CVI-pct-cat-Health.csv")

# take a look at the dataset
CVI_pct_cat_Health.head()

,row#,Name,FIPS,Source,ToxPi Score,Overall Physical Health,Mental Health & Deaths of Despair,Chronic Disease,Chronic Disease Prevention,Infectious Diseases,Child & Maternal Health,Access to Care
0,1,"AL, Autauga, 01001020100",1001020100,"+32.4771112,-086.4903033",0.558778,0.694319,0.357869,0.524669,0.284634,0.565672,0.711061,0.773219
1,2,"AL, Autauga, 01001020200",1001020200,"+32.4757580,-086.4724678",0.596873,0.680967,0.380152,0.651487,0.428084,0.565672,0.711061,0.760689
2,3,"AL, Autauga, 01001020300",1001020300,"+32.4740243,-086.4597033",0.588483,0.732206,0.387528,0.641407,0.384057,0.565672,0.711061,0.697452
3,4,"AL, Autauga, 01001020400",1001020400,"+32.4710782,-086.4446805",0.528484,0.596631,0.286939,0.603796,0.293247,0.565672,0.711061,0.642043
4,5,"AL, Autauga, 01001020500",1001020500,"+32.4589157,-086.4218165",0.448925,0.359547,0.331398,0.360919,0.218258,0.565672,0.711061,0.595623


In [9]:
# THIS DATA GIVES DETAILED INFORMATION ABOUT THE INDICATORS FOR EACH CATEGORY
# Importing the data from the file
CVI_pct_cat_Infrastructure = pd.read_csv("/Users/juanvasquez/Documents/Work/Work_Juan/Infinite_Analytics/Projects/Data_projects/Wellness_index/CVI-main-Vasquez/CVI-pct/CVI-pct-cat-Infrastructure.csv")

# take a look at the dataset
CVI_pct_cat_Infrastructure.head()

,row#,Name,FIPS,Source,ToxPi Score,Transportation,Energy,"Food, Water, and Waste Management",Communications,Financial Services,Governance
0,1,"AL, Autauga, 01001020100",1001020100,"+32.4771112,-086.4903033",0.678387,0.807779,0.683236,0.732635,0.713806,0.667885,0.464980
1,2,"AL, Autauga, 01001020200",1001020200,"+32.4757580,-086.4724678",0.666757,0.854111,0.553729,0.722670,0.650379,0.782804,0.436847
2,3,"AL, Autauga, 01001020300",1001020300,"+32.4740243,-086.4597033",0.663062,0.733421,0.762699,0.725392,0.667554,0.650594,0.438715
3,4,"AL, Autauga, 01001020400",1001020400,"+32.4710782,-086.4446805",0.594168,0.590413,0.679750,0.744700,0.542631,0.548061,0.459451
4,5,"AL, Autauga, 01001020500",1001020500,"+32.4589157,-086.4218165",0.599512,0.585483,0.508225,0.757804,0.266896,0.946107,0.532558


In [10]:
# THIS DATA GIVES DETAILED INFORMATION ABOUT THE INDICATORS FOR EACH CATEGORY
# Importing the data from the file
CVI_pct_cat_Social_Economic = pd.read_csv("/Users/juanvasquez/Documents/Work/Work_Juan/Infinite_Analytics/Projects/Data_projects/Wellness_index/CVI-main-Vasquez/CVI-pct/CVI-pct-cat-Social & Economic.csv")

# take a look at the dataset
CVI_pct_cat_Social_Economic.head()

,row#,Name,FIPS,Source,ToxPi Score,Socioeconomic Stressors,Housing Composition & Disability,Minority Status & Language,Housing Type & Transportation
0,1,"AL, Autauga, 01001020100",1001020100,"+32.4771112,-086.4903033",0.403817,0.521542,0.453149,0.294139,0.346439
1,2,"AL, Autauga, 01001020200",1001020200,"+32.4757580,-086.4724678",0.480832,0.573475,0.389598,0.351394,0.608862
2,3,"AL, Autauga, 01001020300",1001020300,"+32.4740243,-086.4597033",0.504594,0.618527,0.571712,0.369439,0.458700
3,4,"AL, Autauga, 01001020400",1001020400,"+32.4710782,-086.4446805",0.380242,0.395756,0.535134,0.291865,0.298211
4,5,"AL, Autauga, 01001020500",1001020500,"+32.4589157,-086.4218165",0.451523,0.445741,0.480526,0.306850,0.572976


In [11]:
# THIS DATA GIVES DETAILED INFORMATION ABOUT THE INDICATORS FOR EACH CATEGORY
# Importing the data from the file
CVI_pct_cat_Environment = pd.read_csv("/Users/juanvasquez/Documents/Work/Work_Juan/Infinite_Analytics/Projects/Data_projects/Wellness_index/CVI-main-Vasquez/CVI-pct/CVI-pct-cat-Environment.csv")

# take a look at the dataset
CVI_pct_cat_Environment.head()

,row#,Name,FIPS,Source,ToxPi Score,Transportation Sources,Exposures & Risks,Pollution Sources,Criteria Air Pollutants,Land Use
0,1,"AL, Autauga, 01001020100",1001020100,"+32.4771112,-086.4903033",0.366661,0.419585,0.483148,0.273028,0.492274,0.165270
1,2,"AL, Autauga, 01001020200",1001020200,"+32.4757580,-086.4724678",0.390184,0.313628,0.545089,0.288910,0.534175,0.269121
2,3,"AL, Autauga, 01001020300",1001020300,"+32.4740243,-086.4597033",0.419794,0.439315,0.519067,0.317235,0.548805,0.274546
3,4,"AL, Autauga, 01001020400",1001020400,"+32.4710782,-086.4446805",0.430433,0.447274,0.529765,0.344483,0.552791,0.277850
4,5,"AL, Autauga, 01001020500",1001020500,"+32.4589157,-086.4218165",0.424203,0.252926,0.551128,0.380172,0.582514,0.354276


In [12]:
# Importing the data from the file
# THIS IS THE IMPORTANT DATAFRAME SINCE IT CONTAINS THE BASELINE DATA
CVI_pct_comb_baseline = pd.read_csv("./CVI-pct-comb-baseline.csv")

# take a look at the dataset
CVI_pct_comb_baseline.head()

,row#,Name,FIPS,Source,ToxPi Score,Baseline.Health,Baseline.SocialEconomic,Baseline.Infrastructure,Baseline.Environment
0,1,"AL, Autauga, 01001020100",1001020100,"+32.4771112,-086.4903033",0.513116,0.587969,0.358101,0.724892,0.381501
1,2,"AL, Autauga, 01001020200",1001020200,"+32.4757580,-086.4724678",0.556438,0.640211,0.465116,0.708693,0.411731
2,3,"AL, Autauga, 01001020300",1001020300,"+32.4740243,-086.4597033",0.570042,0.628706,0.498134,0.703547,0.449783
3,4,"AL, Autauga, 01001020400",1001020400,"+32.4710782,-086.4446805",0.485703,0.546426,0.325342,0.607588,0.463455
4,5,"AL, Autauga, 01001020500",1001020500,"+32.4589157,-086.4218165",0.483049,0.437324,0.424390,0.615032,0.455449


Here comes the merging of the dataframes with the ToxPi score for each subcategory

In [13]:
columns_Health = ['FIPS', 'Overall Physical Health', 'Mental Health & Deaths of Despair',
       'Chronic Disease', 'Chronic Disease Prevention', 'Infectious Diseases',
       'Child & Maternal Health', 'Access to Care']
columns_Infrastructure = ['FIPS', 'Transportation',
       'Energy', 'Food, Water, and Waste Management', 'Communications',
       'Financial Services', 'Governance']
columns_Social_Economic = ['FIPS',
'Socioeconomic Stressors', 'Housing Composition & Disability',
       'Minority Status & Language', 'Housing Type & Transportation']
columns_Environment = ['FIPS','Transportation Sources', 'Exposures & Risks', 'Pollution Sources',
       'Criteria Air Pollutants', 'Land Use']

######### HEALTH #########
other_df_Health = CVI_pct_cat_Health[columns_Health]

# Create a dictionary with old column names as keys and new column names as values
rename_dict_Health = {col: f"{col} (Baseline.Health)" for col in columns_Health if col != 'FIPS'}

# Rename the columns
other_df_Health = other_df_Health.rename(columns=rename_dict_Health)

######### Infrastructure #########
other_df_Infrastructure = CVI_pct_cat_Infrastructure[columns_Infrastructure]

# Create a dictionary with old column names as keys and new column names as values
rename_dict_Infrastructure = {col: f"{col} (Baseline.Infrastructure)" for col in columns_Infrastructure if col != 'FIPS'}

# Rename the columns
other_df_Infrastructure = other_df_Infrastructure.rename(columns=rename_dict_Infrastructure)

######### Social Economic #########
other_df_Social_Economic = CVI_pct_cat_Social_Economic[columns_Social_Economic]

# Create a dictionary with old column names as keys and new column names as values
rename_dict_Social_Economic = {col: f"{col} (Baseline.Social_Economic)" for col in columns_Social_Economic if col != 'FIPS'}

# Rename the columns
other_df_Social_Economic = other_df_Social_Economic.rename(columns=rename_dict_Social_Economic)

######### Environment #########
other_df_Environment = CVI_pct_cat_Environment[columns_Environment]

# Create a dictionary with old column names as keys and new column names as values
rename_dict_Environment = {col: f"{col} (Baseline.Environment)" for col in columns_Environment if col != 'FIPS'}

# Rename the columns
other_df_Environment = other_df_Environment.rename(columns=rename_dict_Environment)



# Merge the dataframes
CVI_pct_comb_baseline = pd.merge(CVI_pct_comb_baseline, other_df_Health, on='FIPS')
CVI_pct_comb_baseline = pd.merge(CVI_pct_comb_baseline, other_df_Infrastructure, on='FIPS')
CVI_pct_comb_baseline = pd.merge(CVI_pct_comb_baseline, other_df_Social_Economic, on='FIPS')
CVI_pct_comb_baseline = pd.merge(CVI_pct_comb_baseline, other_df_Environment, on='FIPS')
CVI_pct_comb_baseline.head()

,row#,Name,FIPS,Source,ToxPi Score,Baseline.Health,Baseline.SocialEconomic,Baseline.Infrastructure,Baseline.Environment,Overall Physical Health (Baseline.Health),...,Governance (Baseline.Infrastructure),Socioeconomic Stressors (Baseline.Social_Economic),Housing Composition & Disability (Baseline.Social_Economic),Minority Status & Language (Baseline.Social_Economic),Housing Type & Transportation (Baseline.Social_Economic),Transportation Sources (Baseline.Environment),Exposures & Risks (Baseline.Environment),Pollution Sources (Baseline.Environment),Criteria Air Pollutants (Baseline.Environment),Land Use (Baseline.Environment)
0,1,"AL, Autauga, 01001020100",1001020100,"+32.4771112,-086.4903033",0.513116,0.587969,0.358101,0.724892,0.381501,0.694319,...,0.464980,0.521542,0.453149,0.294139,0.346439,0.419585,0.483148,0.273028,0.492274,0.165270
1,2,"AL, Autauga, 01001020200",1001020200,"+32.4757580,-086.4724678",0.556438,0.640211,0.465116,0.708693,0.411731,0.680967,...,0.436847,0.573475,0.389598,0.351394,0.608862,0.313628,0.545089,0.288910,0.534175,0.269121
2,3,"AL, Autauga, 01001020300",1001020300,"+32.4740243,-086.4597033",0.570042,0.628706,0.498134,0.703547,0.449783,0.732206,...,0.438715,0.618527,0.571712,0.369439,0.458700,0.439315,0.519067,0.317235,0.548805,0.274546
3,4,"AL, Autauga, 01001020400",1001020400,"+32.4710782,-086.4446805",0.485703,0.546426,0.325342,0.607588,0.463455,0.596631,...,0.459451,0.395756,0.535134,0.291865,0.298211,0.447274,0.529765,0.344483,0.552791,0.277850
4,5,"AL, Autauga, 01001020500",1001020500,"+32.4589157,-086.4218165",0.483049,0.437324,0.424390,0.615032,0.455449,0.359547,...,0.532558,0.445741,0.480526,0.306850,0.572976,0.252926,0.551128,0.380172,0.582514,0.354276


Next I combine all this subcategories data into the CVI_pct_comb_baseline dataframe 

Understanding the FIPS  

https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt

Below there is an example

  county-level FIPS code      |  place name |
  ---------------| ----------|
   01001      |  Autauga County |
  |   06029    |    Kern County |



  - The column FIPS of CVI_pct_comb_baseline dataframe has the values of the GEOID.Tract column of the dataframe 
  CVI_data_current. For some reason in CVI_pct_comb_baseline there is am additional zero in the Name column in the tract number. The reason is for all FIPS to have the same number of digits
  - These values foe the FIPS do not match with the ones here https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt


 These are the lines codes in get_cvi_toxpi.R 
 
cvi.df<-fread("CVI_data_current.csv",
              keepLeadingZeros = TRUE,integer64 = "numeric")

 Name=paste0(cvi.df$STATE,", ",cvi.df$County_Name,", ",cvi.df$GEOID.Tract),
 
  FIPS=cvi.df$GEOID.Tract, # Not sure if this is the correct FIPS (JCV) 

Using H3 python library to convert latitude and longitude to H3 index

In [14]:
from h3 import h3 as l3lib

lat, lng = +32.4771112,-086.4903033
resolution = 9
l3lib.geo_to_h3(lat, lng, resolution)

'8944ec60bb3ffff'

The H3 library by Uber is used for geospatial indexing. It divides the Earth into hexagonal cells and each cell is assigned a unique index. The function geo_to_h3 converts a latitude and longitude into a H3 index.

The H3 index is a 64 bit integer, but it's often represented as a hexadecimal string for readability. The string '8944ec60bb3ffff' is the H3 index of the cell that contains the point with the latitude and longitude you provided.

The first character of the index, 8, represents the resolution of the cell, which in your case is 9. The rest of the index is a unique identifier for the cell at that resolution.

In [15]:
# Split the 'Source' column into two new columns 'Latitude' and 'Longitude'
CVI_pct_comb_baseline[['Latitude', 'Longitude']] = CVI_pct_comb_baseline['Source'].str.split(',', expand=True)

# Convert the 'Latitude' and 'Longitude' columns to numeric
CVI_pct_comb_baseline['Latitude'] = pd.to_numeric(CVI_pct_comb_baseline['Latitude'])
CVI_pct_comb_baseline['Longitude'] = pd.to_numeric(CVI_pct_comb_baseline['Longitude'])
CVI_pct_comb_baseline.head()

,row#,Name,FIPS,Source,ToxPi Score,Baseline.Health,Baseline.SocialEconomic,Baseline.Infrastructure,Baseline.Environment,Overall Physical Health (Baseline.Health),...,Housing Composition & Disability (Baseline.Social_Economic),Minority Status & Language (Baseline.Social_Economic),Housing Type & Transportation (Baseline.Social_Economic),Transportation Sources (Baseline.Environment),Exposures & Risks (Baseline.Environment),Pollution Sources (Baseline.Environment),Criteria Air Pollutants (Baseline.Environment),Land Use (Baseline.Environment),Latitude,Longitude
0,1,"AL, Autauga, 01001020100",1001020100,"+32.4771112,-086.4903033",0.513116,0.587969,0.358101,0.724892,0.381501,0.694319,...,0.453149,0.294139,0.346439,0.419585,0.483148,0.273028,0.492274,0.165270,32.477111,-86.490303
1,2,"AL, Autauga, 01001020200",1001020200,"+32.4757580,-086.4724678",0.556438,0.640211,0.465116,0.708693,0.411731,0.680967,...,0.389598,0.351394,0.608862,0.313628,0.545089,0.288910,0.534175,0.269121,32.475758,-86.472468
2,3,"AL, Autauga, 01001020300",1001020300,"+32.4740243,-086.4597033",0.570042,0.628706,0.498134,0.703547,0.449783,0.732206,...,0.571712,0.369439,0.458700,0.439315,0.519067,0.317235,0.548805,0.274546,32.474024,-86.459703
3,4,"AL, Autauga, 01001020400",1001020400,"+32.4710782,-086.4446805",0.485703,0.546426,0.325342,0.607588,0.463455,0.596631,...,0.535134,0.291865,0.298211,0.447274,0.529765,0.344483,0.552791,0.277850,32.471078,-86.444681
4,5,"AL, Autauga, 01001020500",1001020500,"+32.4589157,-086.4218165",0.483049,0.437324,0.424390,0.615032,0.455449,0.359547,...,0.480526,0.306850,0.572976,0.252926,0.551128,0.380172,0.582514,0.354276,32.458916,-86.421817


In [16]:
# Adding the H3 index to the dataframe
resolution = 9
CVI_pct_comb_baseline['h3index'] = CVI_pct_comb_baseline.apply(lambda row: l3lib.geo_to_h3(row['Latitude'], row['Longitude'], resolution), axis=1)
CVI_pct_comb_baseline.head()

,row#,Name,FIPS,Source,ToxPi Score,Baseline.Health,Baseline.SocialEconomic,Baseline.Infrastructure,Baseline.Environment,Overall Physical Health (Baseline.Health),...,Minority Status & Language (Baseline.Social_Economic),Housing Type & Transportation (Baseline.Social_Economic),Transportation Sources (Baseline.Environment),Exposures & Risks (Baseline.Environment),Pollution Sources (Baseline.Environment),Criteria Air Pollutants (Baseline.Environment),Land Use (Baseline.Environment),Latitude,Longitude,h3index
0,1,"AL, Autauga, 01001020100",1001020100,"+32.4771112,-086.4903033",0.513116,0.587969,0.358101,0.724892,0.381501,0.694319,...,0.294139,0.346439,0.419585,0.483148,0.273028,0.492274,0.165270,32.477111,-86.490303,8944ec60bb3ffff
1,2,"AL, Autauga, 01001020200",1001020200,"+32.4757580,-086.4724678",0.556438,0.640211,0.465116,0.708693,0.411731,0.680967,...,0.351394,0.608862,0.313628,0.545089,0.288910,0.534175,0.269121,32.475758,-86.472468,8944ec60bcbffff
2,3,"AL, Autauga, 01001020300",1001020300,"+32.4740243,-086.4597033",0.570042,0.628706,0.498134,0.703547,0.449783,0.732206,...,0.369439,0.458700,0.439315,0.519067,0.317235,0.548805,0.274546,32.474024,-86.459703,8944ec60a67ffff
3,4,"AL, Autauga, 01001020400",1001020400,"+32.4710782,-086.4446805",0.485703,0.546426,0.325342,0.607588,0.463455,0.596631,...,0.291865,0.298211,0.447274,0.529765,0.344483,0.552791,0.277850,32.471078,-86.444681,8944ec619abffff
4,5,"AL, Autauga, 01001020500",1001020500,"+32.4589157,-086.4218165",0.483049,0.437324,0.424390,0.615032,0.455449,0.359547,...,0.306850,0.572976,0.252926,0.551128,0.380172,0.582514,0.354276,32.458916,-86.421817,8944ec61807ffff


In [17]:
# Split the 'Name' column into three new columns 'State', 'County', and 'Code'
CVI_pct_comb_baseline[['State', 'County', 'Code']] = CVI_pct_comb_baseline['Name'].str.split(',', expand=True)

# Strip leading and trailing whitespace
CVI_pct_comb_baseline['State'] = CVI_pct_comb_baseline['State'].str.strip()
CVI_pct_comb_baseline['County'] = CVI_pct_comb_baseline['County'].str.strip()
CVI_pct_comb_baseline['Code'] = CVI_pct_comb_baseline['Code'].str.strip()
CVI_pct_comb_baseline

,row#,Name,FIPS,Source,ToxPi Score,Baseline.Health,Baseline.SocialEconomic,Baseline.Infrastructure,Baseline.Environment,Overall Physical Health (Baseline.Health),...,Exposures & Risks (Baseline.Environment),Pollution Sources (Baseline.Environment),Criteria Air Pollutants (Baseline.Environment),Land Use (Baseline.Environment),Latitude,Longitude,h3index,State,County,Code
0,1,"AL, Autauga, 01001020100",1001020100,"+32.4771112,-086.4903033",0.513116,0.587969,0.358101,0.724892,0.381501,0.694319,...,0.483148,0.273028,0.492274,0.165270,32.477111,-86.490303,8944ec60bb3ffff,AL,Autauga,01001020100
1,2,"AL, Autauga, 01001020200",1001020200,"+32.4757580,-086.4724678",0.556438,0.640211,0.465116,0.708693,0.411731,0.680967,...,0.545089,0.288910,0.534175,0.269121,32.475758,-86.472468,8944ec60bcbffff,AL,Autauga,01001020200
2,3,"AL, Autauga, 01001020300",1001020300,"+32.4740243,-086.4597033",0.570042,0.628706,0.498134,0.703547,0.449783,0.732206,...,0.519067,0.317235,0.548805,0.274546,32.474024,-86.459703,8944ec60a67ffff,AL,Autauga,01001020300
3,4,"AL, Autauga, 01001020400",1001020400,"+32.4710782,-086.4446805",0.485703,0.546426,0.325342,0.607588,0.463455,0.596631,...,0.529765,0.344483,0.552791,0.277850,32.471078,-86.444681,8944ec619abffff,AL,Autauga,01001020400
4,5,"AL, Autauga, 01001020500",1001020500,"+32.4589157,-086.4218165",0.483049,0.437324,0.424390,0.615032,0.455449,0.359547,...,0.551128,0.380172,0.582514,0.354276,32.458916,-86.421817,8944ec61807ffff,AL,Autauga,01001020500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73052,73053,"WY, Washakie, 56043000200",56043000200,"+43.8788305,-107.6690520",0.381064,0.414990,0.470020,0.374094,0.265152,0.410545,...,0.136753,0.177966,0.287741,0.182876,43.878830,-107.669052,89279b2c2afffff,WY,Washakie,56043000200
73053,73054,"WY, Washakie, 56043000301",56043000301,"+44.0143691,-107.9563794",0.380542,0.424419,0.467062,0.450011,0.180675,0.393976,...,0.157463,0.068580,0.144510,0.506221,44.014369,-107.956379,89279b74d4fffff,WY,Washakie,56043000301
73054,73055,"WY, Washakie, 56043000302",56043000302,"+44.0285717,-107.9507941",0.406109,0.463148,0.507464,0.431891,0.221931,0.384851,...,0.154099,0.081602,0.215065,0.356724,44.028572,-107.950794,89279b74dd3ffff,WY,Washakie,56043000302
73055,73056,"WY, Weston, 56045951100",56045951100,"+43.8462133,-104.5700202",0.428767,0.407322,0.514924,0.442666,0.350154,0.513667,...,0.092569,0.329607,0.371953,0.310286,43.846213,-104.570020,8926a5c8147ffff,WY,Weston,56045951100


In [18]:
CVI_pct_comb_baseline.columns

Index(['row#', 'Name', 'FIPS', 'Source', 'ToxPi Score', 'Baseline.Health',
       'Baseline.SocialEconomic', 'Baseline.Infrastructure',
       'Baseline.Environment', 'Overall Physical Health (Baseline.Health)',
       'Mental Health & Deaths of Despair (Baseline.Health)',
       'Chronic Disease (Baseline.Health)',
       'Chronic Disease Prevention (Baseline.Health)',
       'Infectious Diseases (Baseline.Health)',
       'Child & Maternal Health (Baseline.Health)',
       'Access to Care (Baseline.Health)',
       'Transportation (Baseline.Infrastructure)',
       'Energy (Baseline.Infrastructure)',
       'Food, Water, and Waste Management (Baseline.Infrastructure)',
       'Communications (Baseline.Infrastructure)',
       'Financial Services (Baseline.Infrastructure)',
       'Governance (Baseline.Infrastructure)',
       'Socioeconomic Stressors (Baseline.Social_Economic)',
       'Housing Composition & Disability (Baseline.Social_Economic)',
       'Minority Status & Language (

In what follows I will check that the latitude and longitude fall within the 

In [19]:
#  I am using the latitud and longitude to  join the baseline and the shapefile dataframes
# Load your shapefile into a GeoDataFrame
# Replace 'path_to_shapefile' with the actual path to your shapefile
county_shapefile = gpd.read_file('/Users/juanvasquez/Documents/Work/Work_Juan/Infinite_Analytics/Projects/Data_projects/Wellness_index/CVI-main-Vasquez/Data_Visualization/2010 Tracts/2010tracts.shp')
# Print the CRS of the shapefile
print(county_shapefile.crs)
# Convert the 'CVI_pct_comb_baseline' DataFrame to a GeoDataFrame
gdf_points = gpd.GeoDataFrame(
    CVI_pct_comb_baseline, 
    geometry=gpd.points_from_xy(CVI_pct_comb_baseline['Longitude'], CVI_pct_comb_baseline['Latitude']),
    crs="EPSG:4269"  # Ensure this is the same CRS as the shapefile printed above
)

# If necessary, convert the CRS of the points GeoDataFrame to match the county shapefile
gdf_points = gdf_points.to_crs(county_shapefile.crs)

# Perform the spatial join
joined_gdf = gpd.sjoin(gdf_points, county_shapefile, how='inner', op='within')

# Check the resulting joined GeoDataFrame
# It seems all rows are within the limit 
joined_gdf

EPSG:4269


/opt/anaconda3/envs/wellness-score/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,row#,Name,FIPS,Source,ToxPi Score,Baseline.Health,Baseline.SocialEconomic,Baseline.Infrastructure,Baseline.Environment,Overall Physical Health (Baseline.Health),...,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,Shape_Leng,Shape_Area
0,1,"AL, Autauga, 01001020100",1001020100,"+32.4771112,-086.4903033",0.513116,0.587969,0.358101,0.724892,0.381501,0.694319,...,201,Census Tract 201,G5020,S,9.809944e+06,36312.0,+32.4771112,-086.4903033,0.153541,0.000945
1,2,"AL, Autauga, 01001020200",1001020200,"+32.4757580,-086.4724678",0.556438,0.640211,0.465116,0.708693,0.411731,0.680967,...,202,Census Tract 202,G5020,S,3.340505e+06,5846.0,+32.4757580,-086.4724678,0.093097,0.000321
2,3,"AL, Autauga, 01001020300",1001020300,"+32.4740243,-086.4597033",0.570042,0.628706,0.498134,0.703547,0.449783,0.732206,...,203,Census Tract 203,G5020,S,5.349274e+06,9054.0,+32.4740243,-086.4597033,0.100499,0.000514
3,4,"AL, Autauga, 01001020400",1001020400,"+32.4710782,-086.4446805",0.485703,0.546426,0.325342,0.607588,0.463455,0.596631,...,204,Census Tract 204,G5020,S,6.382705e+06,16244.0,+32.4710782,-086.4446805,0.117686,0.000614
4,5,"AL, Autauga, 01001020500",1001020500,"+32.4589157,-086.4218165",0.483049,0.437324,0.424390,0.615032,0.455449,0.359547,...,205,Census Tract 205,G5020,S,1.139772e+07,48412.0,+32.4589157,-086.4218165,0.161868,0.001098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73052,73053,"WY, Washakie, 56043000200",56043000200,"+43.8788305,-107.6690520",0.381064,0.414990,0.470020,0.374094,0.265152,0.410545,...,2,Census Tract 2,G5020,S,5.780556e+09,9805360.0,+43.8788305,-107.6690520,4.567144,0.648710
73053,73054,"WY, Washakie, 56043000301",56043000301,"+44.0143691,-107.9563794",0.380542,0.424419,0.467062,0.450011,0.180675,0.393976,...,3.01,Census Tract 3.01,G5020,S,1.993201e+06,0.0,+44.0143691,-107.9563794,0.062812,0.000224
73054,73055,"WY, Washakie, 56043000302",56043000302,"+44.0285717,-107.9507941",0.406109,0.463148,0.507464,0.431891,0.221931,0.384851,...,3.02,Census Tract 3.02,G5020,S,1.526579e+07,956995.0,+44.0285717,-107.9507941,0.340973,0.001821
73055,73056,"WY, Weston, 56045951100",56045951100,"+43.8462133,-104.5700202",0.428767,0.407322,0.514924,0.442666,0.350154,0.513667,...,9511,Census Tract 9511,G5020,S,6.100219e+09,4817922.0,+43.8462133,-104.5700202,4.016400,0.683236


Understanding resolution and granularoty of the data

In [20]:
def find_hexagon_area_for_resolution(latitude, longitude, resolution):
    # Generate the H3 index for the given resolution
    h3_index = h3.geo_to_h3(latitude, longitude, resolution)
    
    # Calculate the area in square meters
    hexagon_area_m2 = h3.hex_area(resolution, unit='m^2')
    
    return hexagon_area_m2

# Example usage:
latitude = 42.3601  # Replace with your actual latitude
longitude = -71.0589  # Replace with your actual longitude

for res in range(16):  # Loop through resolutions 0 to 15
    area = find_hexagon_area_for_resolution(latitude, longitude, res)
    print(f"Resolution {res} yields a hexagon area of {area} square meters")

Resolution 0 yields a hexagon area of 4250546848000.0 square meters
Resolution 1 yields a hexagon area of 607220978200.0 square meters
Resolution 2 yields a hexagon area of 86745854030.0 square meters
Resolution 3 yields a hexagon area of 12392264860.0 square meters
Resolution 4 yields a hexagon area of 1770323552.0 square meters
Resolution 5 yields a hexagon area of 252903364.5 square meters
Resolution 6 yields a hexagon area of 36129052.1 square meters
Resolution 7 yields a hexagon area of 5161293.2 square meters
Resolution 8 yields a hexagon area of 737327.6 square meters
Resolution 9 yields a hexagon area of 105332.5 square meters
Resolution 10 yields a hexagon area of 15047.5 square meters
Resolution 11 yields a hexagon area of 2149.6000000000004 square meters
Resolution 12 yields a hexagon area of 307.09999999999997 square meters
Resolution 13 yields a hexagon area of 43.900000000000006 square meters
Resolution 14 yields a hexagon area of 6.3 square meters
Resolution 15 yields a 

In [21]:
def find_hexagon_edge_length_for_resolution(latitude, longitude, resolution):
    # Generate the H3 index for the given resolution
    h3_index = h3.geo_to_h3(latitude, longitude, resolution)
    
    # Calculate the edge length in meters
    hexagon_edge_length_m = h3.edge_length(resolution, unit='m')
    
    return hexagon_edge_length_m

# Example usage:
latitude = 42.3601  # Replace with your actual latitude
longitude = -71.0589  # Replace with your actual longitude

for res in range(16):  # Loop through resolutions 0 to 15
    edge_length = find_hexagon_edge_length_for_resolution(latitude, longitude, res)
    print(f"Resolution {res} yields a hexagon with edge length of approximately {edge_length:.2f} meters")


Resolution 0 yields a hexagon with edge length of approximately 1107712.59 meters
Resolution 1 yields a hexagon with edge length of approximately 418676.01 meters
Resolution 2 yields a hexagon with edge length of approximately 158244.66 meters
Resolution 3 yields a hexagon with edge length of approximately 59810.86 meters
Resolution 4 yields a hexagon with edge length of approximately 22606.38 meters
Resolution 5 yields a hexagon with edge length of approximately 8544.41 meters
Resolution 6 yields a hexagon with edge length of approximately 3229.48 meters
Resolution 7 yields a hexagon with edge length of approximately 1220.63 meters
Resolution 8 yields a hexagon with edge length of approximately 461.35 meters
Resolution 9 yields a hexagon with edge length of approximately 174.38 meters
Resolution 10 yields a hexagon with edge length of approximately 65.91 meters
Resolution 11 yields a hexagon with edge length of approximately 24.91 meters
Resolution 12 yields a hexagon with edge length

Calculate the distance between points to to determine the granularity of the data

In [22]:
# Calculate distances between consecutive points
distances = []
for i in range(len(CVI_pct_comb_baseline) - 1):
    point1 = (CVI_pct_comb_baseline.iloc[i]['Latitude'], CVI_pct_comb_baseline.iloc[i]['Longitude'])
    point2 = (CVI_pct_comb_baseline.iloc[i + 1]['Latitude'], CVI_pct_comb_baseline.iloc[i + 1]['Longitude'])
    distance = geodesic(point1, point2).meters  # distance in meters
    distances.append(distance)

# Append distance data to the DataFrame
CVI_pct_comb_baseline['Distance_to_next_point'] = pd.Series(distances + [None])  # Append None for the last row

# Print DataFrame to verify
CVI_pct_comb_baseline[['Name', 'Latitude', 'Longitude', 'Distance_to_next_point']].head()

,Name,Latitude,Longitude,Distance_to_next_point
0,"AL, Autauga, 01001020100",32.477111,-86.490303,1683.261894
1,"AL, Autauga, 01001020200",32.475758,-86.472468,1215.204677
2,"AL, Autauga, 01001020300",32.474024,-86.459703,1449.521036
3,"AL, Autauga, 01001020400",32.471078,-86.444681,2537.631818
4,"AL, Autauga, 01001020500",32.458916,-86.421817,5326.859471


Analyzing Distance Distribution

In [23]:
if distances:
    average_distance = sum(distances) / len(distances)
    print(f"Average distance between points: {average_distance} meters")
    print(f"Minimum distance: {min(distances)} meters")
    print(f"Maximum distance: {max(distances)} meters")


Average distance between points: 18325.50395703666 meters
Minimum distance: 132.65650362667964 meters
Maximum distance: 7157506.074058095 meters


In [24]:
min_distance_index = CVI_pct_comb_baseline['Distance_to_next_point'].idxmin()
min_distance_name = CVI_pct_comb_baseline.loc[min_distance_index,['Name','Distance_to_next_point']]
min_distance_name

Name                      NY, Bronx, 36005023501
Distance_to_next_point                132.656504
Name: 43446, dtype: object

In [25]:
max_distance_index = CVI_pct_comb_baseline['Distance_to_next_point'].idxmax()
max_distance_name = CVI_pct_comb_baseline.loc[max_distance_index, ['Name','Distance_to_next_point']]
max_distance_name

Name                      GA, Worth, 13321950600
Distance_to_next_point            7157506.074058
Name: 20309, dtype: object

In [26]:
# Calculate distances between consecutive points within each state
def calculate_distances(df):
    distances = []
    for i in range(len(df) - 1):
        point1 = (df.iloc[i]['Latitude'], df.iloc[i]['Longitude'])
        point2 = (df.iloc[i + 1]['Latitude'], df.iloc[i + 1]['Longitude'])
        distance = geodesic(point1, point2).meters  # distance in meters
        distances.append(distance)
    return distances

# Apply the function to each group and create a new column for distances
CVI_pct_comb_baseline['Distance_to_next_point'] = None
for state, group in CVI_pct_comb_baseline.groupby('State'):
    indices = group.index[:-1]  # Exclude the last index because there's no next point
    distances = calculate_distances(group)
    CVI_pct_comb_baseline.loc[indices, 'Distance_to_next_point'] = distances

# Group by state and calculate the average distance
average_distances = CVI_pct_comb_baseline.groupby('State')['Distance_to_next_point'].mean()

# Print the average distances for each state
average_distances

State
AK    163269.50456
AL    20042.227408
AR     32178.56781
AZ     12596.65632
CA     7387.971398
CO    19707.579471
CT     5474.814123
DC     1592.880305
DE     6131.327287
FL    10104.729925
GA    22535.887912
HI     22891.92356
IA    32346.982011
ID    60574.863743
IL     12022.15444
IN    16724.214706
KS    43223.976061
KY    27408.296274
LA    18816.578362
MA      5043.54795
MD     6409.020165
ME    19869.937267
MI    14190.427747
MN     24606.43961
MO    26509.838064
MS    32431.885897
MT    91283.215486
NC    17446.558199
ND    75519.933686
NE    49017.637555
NH    11288.106278
NJ     4858.276369
NM    33189.783723
NV    15664.039844
NY     8359.855172
OH    10958.091173
OK    23971.670106
OR    19429.269259
PA     8968.802167
RI     4784.462575
SC    13935.429132
SD    85689.337616
TN    21056.170691
TX    27365.421829
UT    19352.810034
VA     19863.49232
VT    17501.461044
WA    13843.496549
WI    18557.809447
WV    27236.154572
WY     70138.97339
Name: Distance_to_next_po

In [27]:
import operator

# Sort the average_distances dictionary by value
sorted_average_distances = sorted(average_distances.items(), key=operator.itemgetter(1))
sorted_average_distances


[('DC', 1592.8803047910412),
 ('RI', 4784.462574704528),
 ('NJ', 4858.276368723594),
 ('MA', 5043.547949511351),
 ('CT', 5474.81412273245),
 ('DE', 6131.327287491842),
 ('MD', 6409.0201646397745),
 ('CA', 7387.971397768964),
 ('NY', 8359.855171527483),
 ('PA', 8968.802166598833),
 ('FL', 10104.72992508669),
 ('OH', 10958.09117295913),
 ('NH', 11288.10627847921),
 ('IL', 12022.154440375698),
 ('AZ', 12596.65631980322),
 ('WA', 13843.496548774054),
 ('SC', 13935.429132259782),
 ('MI', 14190.42774664489),
 ('NV', 15664.039843831311),
 ('IN', 16724.214705893697),
 ('NC', 17446.55819931843),
 ('VT', 17501.46104427167),
 ('WI', 18557.809447041076),
 ('LA', 18816.578361684922),
 ('UT', 19352.810034216447),
 ('OR', 19429.269258937242),
 ('CO', 19707.57947099237),
 ('VA', 19863.492320326954),
 ('ME', 19869.93726725888),
 ('AL', 20042.227407534036),
 ('TN', 21056.17069056546),
 ('GA', 22535.887912075417),
 ('HI', 22891.92356007236),
 ('OK', 23971.670105657216),
 ('MN', 24606.439609571033),
 ('MO

The next step is to figure out which locations have which granurality. 

Writing the joined dataframe into a parquet file

In [28]:
# Write the DataFrame to a parquet file for future use
joined_gdf.to_parquet('joined_gdf.parquet')

In [29]:
joined_gdf.columns

Index(['row#', 'Name', 'FIPS', 'Source', 'ToxPi Score', 'Baseline.Health',
       'Baseline.SocialEconomic', 'Baseline.Infrastructure',
       'Baseline.Environment', 'Overall Physical Health (Baseline.Health)',
       'Mental Health & Deaths of Despair (Baseline.Health)',
       'Chronic Disease (Baseline.Health)',
       'Chronic Disease Prevention (Baseline.Health)',
       'Infectious Diseases (Baseline.Health)',
       'Child & Maternal Health (Baseline.Health)',
       'Access to Care (Baseline.Health)',
       'Transportation (Baseline.Infrastructure)',
       'Energy (Baseline.Infrastructure)',
       'Food, Water, and Waste Management (Baseline.Infrastructure)',
       'Communications (Baseline.Infrastructure)',
       'Financial Services (Baseline.Infrastructure)',
       'Governance (Baseline.Infrastructure)',
       'Socioeconomic Stressors (Baseline.Social_Economic)',
       'Housing Composition & Disability (Baseline.Social_Economic)',
       'Minority Status & Language (

## Internal point of interest (poi) data (Under development)

In [30]:
# Importing the data from the file
poi_df = pd.read_csv("./sqllab_wbiusgetdata_20240419T201502.csv")

# take a look at the dataset
poi_df.head(20)

,poi_type,district_code,h3index,reviews_count,ratings,prices,validation_source_url,open_close_hours,h3resolution,quality,brands,building_type,latitude,longitude,population_density,population_density_percentile,district,state,sname,country
0,shop.sports.hunt_fish,2122,890c42c0173ffff,7.0,4.0,[],https://www.google.com/maps/place/Inn+and+Out+...,"[4.6, 7]",9.0,-1.0,NaN,NaN,60.126500,-149.437138,58.631244,0.910596,Kenai Peninsula Borough,Alaska,Hunting and Fishing Shop,USA
1,shop.home.improvement,2020,890c7338bd7ffff,4.0,4.0,[],NaN,"[4, 4]",9.0,-1.0,NaN,NaN,61.131594,-149.877831,0.000000,0.040844,Anchorage Municipality,Alaska,Home Improvement Store,USA
2,health.clinic.physician,2170,890c73ba51bffff,24.0,3.0,[],https://www.google.com/maps/place/Pioneer+Peak...,"[3.9, 24]",9.0,-1.0,NaN,NaN,61.572799,-149.354046,17.753839,0.821661,Matanuska-Susitna Borough,Alaska,Physician,USA
3,health.treatment.physical,2170,890c73ba51bffff,161.0,5.0,[],NaN,"[5, 161]",9.0,-1.0,NaN,NaN,61.572799,-149.354046,17.753839,0.821661,Matanuska-Susitna Borough,Alaska,Physical Therapist,USA
4,shop.sports.kayak,2170,890c73c6e07ffff,5.0,5.0,[],NaN,"[5, 5]",9.0,-1.0,NaN,NaN,61.662094,-150.107370,0.000000,NaN,Matanuska-Susitna Borough,Alaska,Canoe-Kayak Shop,USA
5,travel.attraction.animal_park,2170,890c73c6e07ffff,123.0,4.0,[],https://www.google.com/maps/place/Nancy+Lake+S...,"[4.4, 123]",9.0,-1.0,NaN,NaN,61.662094,-150.107370,0.000000,NaN,Matanuska-Susitna Borough,Alaska,Animal Park,USA
6,travel.accommodation.lodging,2164,890ce170437ffff,4.0,5.0,[],https://www.google.com/maps/place/Alagnak+Lodg...,"[5, 4]",9.0,-1.0,NaN,NaN,59.042664,-156.721847,0.000000,NaN,Lake and Peninsula Borough,Alaska,Lodging,USA
7,shop.sports.outdoor,2195,8912b49a877ffff,4.0,4.0,[],https://www.google.com/maps/place/Hungry+Pt.+T...,"[4.3, 4]",9.0,-1.0,NaN,NaN,56.814906,-132.942595,0.000000,NaN,Petersburg Borough,Alaska,Outdoor Shop,USA
8,service.maintenance.electrical,53019,8912da121abffff,11.0,3.0,[],https://www.google.com/maps/place/Preferred+El...,"[3.8, 11]",9.0,-1.0,NaN,NaN,48.503794,-118.735829,0.000000,NaN,Ferry County,Washington,Electrical Service,USA
9,shop.sports.outdoor,53065,8912dab2cabffff,0.0,0.0,[],https://www.google.com/maps/place/Kettle+Falls...,[],9.0,-1.0,NaN,NaN,48.597616,-118.119536,29.187215,0.865876,Stevens County,Washington,Outdoor Shop,USA


In [31]:
# Getting information about the dataset
poi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   poi_type                       100000 non-null  object 
 1   district_code                  100000 non-null  int64  
 2   h3index                        97941 non-null   object 
 3   reviews_count                  85554 non-null   float64
 4   ratings                        85554 non-null   float64
 5   prices                         85554 non-null   object 
 6   validation_source_url          67522 non-null   object 
 7   open_close_hours               85554 non-null   object 
 8   h3resolution                   95894 non-null   float64
 9   quality                        13189 non-null   float64
 10  brands                         10733 non-null   object 
 11  building_type                  0 non-null       float64
 12  latitude                       

In [32]:
# Getting the description of the dataset
poi_df.dtypes

poi_type                          object
district_code                      int64
h3index                           object
reviews_count                    float64
ratings                          float64
prices                            object
validation_source_url             object
open_close_hours                  object
h3resolution                     float64
quality                          float64
brands                            object
building_type                    float64
latitude                         float64
longitude                        float64
population_density               float64
population_density_percentile    float64
district                          object
state                             object
sname                             object
country                           object
dtype: object

In [33]:
# Getting the unique values of the dataset for the column 'poi_type'
poi_df['poi_type'].unique()

array(['shop.sports.hunt_fish', 'shop.home.improvement',
       'health.clinic.physician', 'health.treatment.physical',
       'shop.sports.kayak', 'travel.attraction.animal_park',
       'travel.accommodation.lodging', 'shop.sports.outdoor',
       'service.maintenance.electrical', 'leisure.outdoor',
       'health.clinic.dentist', 'facility.education.training',
       'health.treatment.psychological', 'health.clinic.alternative',
       'building.office.consultant', 'health.treatment.diet',
       'service.maintenance.contracting', 'service.maintenance.maid',
       'service.maintenance.construction', 'service.finance.institute',
       'health.treatment.chiropractor', 'service.pet.veterinarian',
       'health.hospital.specialty', 'service.marriage',
       'facility.education.arts', 'shop.home.mattress',
       'health.clinic.health_centre', 'building.office.estate_agent',
       'service.maintenance.plumbing', 'shop.food.butcher',
       'shop.home.bathroom_furnishing', 'leisure.o

## Validation of the scores 

This webpages gives information:

1. https://worldpopulationreview.com/state-rankings/quality-of-life-by-state (see table at the end of this page)
2. https://www.countyhealthrankings.org/findings-and-insights/2024-national-findings-report (for county level info on health and Environment)
3. Distressed Communities Index (https://eig.org/distressed-communities/?geo=counties&lat=38.55&lon=-96.42&z=3.91) (used for socio-economic)
4. Still missing infrastructure validation (to be done)

### Importing Data

In [34]:
# This data helps in validating: 
# 1) Health 
# 2) Environment
# Importing the data from the file for the validation data from https://www.countyhealthrankings.org/health-data/methodology-and-sources/data-documentation
health_county_validation_data = pd.read_excel('./Validation-data/Health_Environment/2024_county_health_release_data_v1.xlsx', sheet_name='Health Outcomes & Factors')
health_county_validation_data['Health Outcomes'].head(20)

0     National Z-Score
1                  NaN
2             0.133257
3            -0.223533
4             1.346811
5             0.825542
6             0.300239
7             1.762632
8             1.237395
9             0.815006
10            1.083606
11            0.530136
12            0.448786
13            1.057633
14            1.507326
15             0.97118
16            0.641422
17            0.276477
18            0.556178
19            2.108428
Name: Health Outcomes, dtype: object

In [35]:
# This data helps in validating: 
# 3) Social & Economic
# Importing the data from the file for the validation data from https://www.ers.usda.gov/data-products/county-level-data-sets/county-level-data-sets-download-data/
soecon_county_validation_data = pd.read_excel('./Validation-data/Social_Economic/PovertyEstimates.xlsx', sheet_name='PovertyEstimates')
#soecon_county_validation_data.drop([0, 1, 2], inplace=True) # Drop the first three rows
soecon_county_validation_data.columns = soecon_county_validation_data.iloc[3]
soecon_county_validation_data = soecon_county_validation_data.drop(3)
soecon_county_validation_data = soecon_county_validation_data.reset_index(drop=True)
soecon_county_validation_data['Area_name'] = soecon_county_validation_data['Area_name'].str.replace('County', '', case=False) # Removing the county word from the Area_name column
soecon_county_validation_data[['Stabr','Area_name','POVALL_2021']].head(20)

3,Stabr,Area_name,POVALL_2021
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,US,United States,41393176
4,AL,Alabama,800848
5,AL,Autauga,6296
6,AL,Baldwin,25526
7,AL,Barbour,5089
8,AL,Bibb,4204
9,AL,Blount,6992


In [36]:
# Environment validation data
# I have to add data from here 
# https://www.countyhealthrankings.org/health-data/massachusetts?year=2024&measure=Drinking+Water+Violations&tab=1
import pandas as pd
import os
import glob

def import_csvs(root_dir, exclude='footnotes.csv'):
    """Import all CSV files in the specified directory and its subdirectories, excluding specified files, and add a 'State' column."""
    # Create a list to store dataframes
    dataframes = []
    
    # Walk through the directory
    for root, dirs, files in os.walk(root_dir):
        # Find all CSV files and read each one, excluding specified filenames
        csv_files = glob.glob(os.path.join(root, '*.csv'))
        for filename in csv_files:
            if os.path.basename(filename).lower() != exclude.lower():
                df = pd.read_csv(filename)
                # Check if 'County' column exists and add 'State' column based on its first value
                if 'County' in df.columns and not df.empty:
                    state_name = df['County'].iloc[0]  # Assumes the state name is in the first row of 'County'
                    df['State'] = state_name
                dataframes.append(df)
                print(f"Loaded {filename} with additional 'State' column.")
    
    return dataframes

# Set the root directory where your CSV files are stored
root_directory = '/Users/juanvasquez/Documents/Work/Work_Juan/Infinite_Analytics/Projects/Data_projects/Wellness_index/CVI-main-Vasquez/Data_Visualization/Validation-data/Environment/scraped_data'#'/path/to/your/directory'

# Call the function and get the list of dataframes
csv_dataframes = import_csvs(root_directory)
# Concatenate all dataframes into a single dataframe
infra_county_validation_data = pd.concat(csv_dataframes, ignore_index=True)
# Remove Parish,	Parish Value,	Borough,	Borough Value columns  from the dataframe
infra_county_validation_data.drop(['Parish', 'Parish Value', 'Borough', 'Borough Value','County Value'], axis=1, inplace=True)
# Chage the order of the columns to State, County and National Z-Score
infra_county_validation_data = infra_county_validation_data[['State', 'County', 'National Z-Score']]
infra_county_validation_data.tail(20)



Loaded /Users/juanvasquez/Documents/Work/Work_Juan/Infinite_Analytics/Projects/Data_projects/Wellness_index/CVI-main-Vasquez/Data_Visualization/Validation-data/Environment/scraped_data/arkansas-2024-air-pollution---particulate-matter-place-sort/arkansas-2024-air-pollution---particulate-matter-place-sort.csv with additional 'State' column.
Loaded /Users/juanvasquez/Documents/Work/Work_Juan/Infinite_Analytics/Projects/Data_projects/Wellness_index/CVI-main-Vasquez/Data_Visualization/Validation-data/Environment/scraped_data/rhode-island-2024-air-pollution---particulate-matter-place-sort/rhode-island-2024-air-pollution---particulate-matter-place-sort.csv with additional 'State' column.
Loaded /Users/juanvasquez/Documents/Work/Work_Juan/Infinite_Analytics/Projects/Data_projects/Wellness_index/CVI-main-Vasquez/Data_Visualization/Validation-data/Environment/scraped_data/tennessee-2024-air-pollution---particulate-matter-place-sort/tennessee-2024-air-pollution---particulate-matter-place-sort.csv

,State,County,National Z-Score
2805,Colorado,Morgan,-0.82
2806,Colorado,Otero,-1.65
2807,Colorado,Ouray,-2.36
2808,Colorado,Park,-2.07
2809,Colorado,Phillips,-1.53
2810,Colorado,Pitkin,-2.30
2811,Colorado,Prowers,-1.29
2812,Colorado,Pueblo,-1.71
2813,Colorado,Rio Blanco,0.19
2814,Colorado,Rio Grande,-2.07


In [37]:
# Infrastructure validation data
# I have to add data from here giving % Households with Broadband Access
# https://www.countyhealthrankings.org/health-data/wyoming?year=2024&measure=Broadband+Access*&tab=1

# Set the root directory where your CSV files are stored
root_directory = '/Users/juanvasquez/Documents/Work/Work_Juan/Infinite_Analytics/Projects/Data_projects/Wellness_index/CVI-main-Vasquez/Data_Visualization/Validation-data/Infrastructure/scraped_data'#'/path/to/your/directory'

# Call the function and get the list of dataframes
csv_dataframes = import_csvs(root_directory)
# Concatenate all dataframes into a single dataframe
infrastructure_county_validation_data = pd.concat(csv_dataframes, ignore_index=True)
# Chage the order of the columns to State, County and National Z-Score
infrastructure_county_validation_data = infrastructure_county_validation_data[['State', 'County','County Value**', '# Households with Broadband Access','Error Margin']]
infrastructure_county_validation_data.head(10)

Loaded /Users/juanvasquez/Documents/Work/Work_Juan/Infinite_Analytics/Projects/Data_projects/Wellness_index/CVI-main-Vasquez/Data_Visualization/Validation-data/Infrastructure/scraped_data/colorado-2024-broadband-access-place-sort/colorado-2024-broadband-access-place-sort.csv with additional 'State' column.
Loaded /Users/juanvasquez/Documents/Work/Work_Juan/Infinite_Analytics/Projects/Data_projects/Wellness_index/CVI-main-Vasquez/Data_Visualization/Validation-data/Infrastructure/scraped_data/new-mexico-2024-broadband-access-place-sort/new-mexico-2024-broadband-access-place-sort.csv with additional 'State' column.
Loaded /Users/juanvasquez/Documents/Work/Work_Juan/Infinite_Analytics/Projects/Data_projects/Wellness_index/CVI-main-Vasquez/Data_Visualization/Validation-data/Infrastructure/scraped_data/west-virginia-2024-broadband-access-place-sort/west-virginia-2024-broadband-access-place-sort.csv with additional 'State' column.
Loaded /Users/juanvasquez/Documents/Work/Work_Juan/Infinite_An

,State,County,County Value**,# Households with Broadband Access,Error Margin
0,Colorado,Colorado,92%,"2,091,383",92-92%
1,Colorado,Adams,91%,"165,088",90-92
2,Colorado,Alamosa,89%,"5,757",86-92
3,Colorado,Arapahoe,94%,"234,735",93-94
4,Colorado,Archuleta,90%,"5,136",86-93
5,Colorado,Baca,81%,"1,280",74-87
6,Colorado,Bent,81%,"1,471",77-86
7,Colorado,Boulder,94%,"125,497",93-95
8,Colorado,Broomfield,95%,"28,585",94-96
9,Colorado,Chaffee,83%,"7,558",79-87


In [38]:
# I will validate health data for each county using the average of the ToxPi score for each county. CVI data 
CVI_pct_comb_baseline.head()

,row#,Name,FIPS,Source,ToxPi Score,Baseline.Health,Baseline.SocialEconomic,Baseline.Infrastructure,Baseline.Environment,Overall Physical Health (Baseline.Health),...,Pollution Sources (Baseline.Environment),Criteria Air Pollutants (Baseline.Environment),Land Use (Baseline.Environment),Latitude,Longitude,h3index,State,County,Code,Distance_to_next_point
0,1,"AL, Autauga, 01001020100",1001020100,"+32.4771112,-086.4903033",0.513116,0.587969,0.358101,0.724892,0.381501,0.694319,...,0.273028,0.492274,0.165270,32.477111,-86.490303,8944ec60bb3ffff,AL,Autauga,01001020100,1683.261894
1,2,"AL, Autauga, 01001020200",1001020200,"+32.4757580,-086.4724678",0.556438,0.640211,0.465116,0.708693,0.411731,0.680967,...,0.288910,0.534175,0.269121,32.475758,-86.472468,8944ec60bcbffff,AL,Autauga,01001020200,1215.204677
2,3,"AL, Autauga, 01001020300",1001020300,"+32.4740243,-086.4597033",0.570042,0.628706,0.498134,0.703547,0.449783,0.732206,...,0.317235,0.548805,0.274546,32.474024,-86.459703,8944ec60a67ffff,AL,Autauga,01001020300,1449.521036
3,4,"AL, Autauga, 01001020400",1001020400,"+32.4710782,-086.4446805",0.485703,0.546426,0.325342,0.607588,0.463455,0.596631,...,0.344483,0.552791,0.277850,32.471078,-86.444681,8944ec619abffff,AL,Autauga,01001020400,2537.631818
4,5,"AL, Autauga, 01001020500",1001020500,"+32.4589157,-086.4218165",0.483049,0.437324,0.424390,0.615032,0.455449,0.359547,...,0.380172,0.582514,0.354276,32.458916,-86.421817,8944ec61807ffff,AL,Autauga,01001020500,5326.859471


In [39]:
# Population data from the US Census Bureau
# https://www2.census.gov/programs-surveys/popest/datasets/2020-2022/counties/totals/co-est2022-alldata.csv
# Importing the data from the file
population_data = pd.read_csv("./Validation-data/Social_Economic/co-est2022-alldata.csv")
# Removing the county word from the Area_name column
population_data['CTYNAME'] = population_data['CTYNAME'].str.replace('County', '', case=False)
population_data.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,...,RDEATH2021,RDEATH2022,RNATURALCHG2021,RNATURALCHG2022,RINTERNATIONALMIG2021,RINTERNATIONALMIG2022,RDOMESTICMIG2021,RDOMESTICMIG2022,RNETMIG2021,RNETMIG2022
0,40,3,6,1,0,Alabama,Alabama,5024356,5031362,5049846,...,13.699945,13.210008,-2.369557,-1.696934,0.358092,0.908126,5.496167,5.651639,5.854259,6.559766
1,50,3,6,1,1,Alabama,Autauga,58802,58902,59210,...,11.548361,11.313872,0.203197,0.605200,0.253996,0.453900,4.063939,8.540040,4.317936,8.993940
2,50,3,6,1,3,Alabama,Baldwin,231761,233219,239361,...,12.882475,11.976220,-2.865123,-2.050243,0.444369,1.296841,29.493419,28.744576,29.937788,30.041417
3,50,3,6,1,5,Alabama,Barbour,25224,24960,24539,...,15.475060,15.108133,-4.323320,-3.858260,0.000000,0.162453,-12.444696,9.869022,-12.444696,10.031475
4,50,3,6,1,7,Alabama,Bibb,22300,22183,22370,...,14.275133,14.467606,-3.321886,-3.290141,0.044890,0.090141,11.626602,-13.566197,11.671492,-13.476056


In [40]:
# Writing STNAME in two letter state code 
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
# Changing the column names so that they match the validation data column names 
population_data.rename(columns={'STNAME': 'Stabr', 'CTYNAME': 'Area_name'}, inplace=True)

population_data['Stabr'] = population_data['Stabr'].map(state_abbreviations)
population_data.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,Stabr,Area_name,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,...,RDEATH2021,RDEATH2022,RNATURALCHG2021,RNATURALCHG2022,RINTERNATIONALMIG2021,RINTERNATIONALMIG2022,RDOMESTICMIG2021,RDOMESTICMIG2022,RNETMIG2021,RNETMIG2022
0,40,3,6,1,0,AL,Alabama,5024356,5031362,5049846,...,13.699945,13.210008,-2.369557,-1.696934,0.358092,0.908126,5.496167,5.651639,5.854259,6.559766
1,50,3,6,1,1,AL,Autauga,58802,58902,59210,...,11.548361,11.313872,0.203197,0.605200,0.253996,0.453900,4.063939,8.540040,4.317936,8.993940
2,50,3,6,1,3,AL,Baldwin,231761,233219,239361,...,12.882475,11.976220,-2.865123,-2.050243,0.444369,1.296841,29.493419,28.744576,29.937788,30.041417
3,50,3,6,1,5,AL,Barbour,25224,24960,24539,...,15.475060,15.108133,-4.323320,-3.858260,0.000000,0.162453,-12.444696,9.869022,-12.444696,10.031475
4,50,3,6,1,7,AL,Bibb,22300,22183,22370,...,14.275133,14.467606,-3.321886,-3.290141,0.044890,0.090141,11.626602,-13.566197,11.671492,-13.476056


### Validation of of the Health Baseline Vulnerability (done)

In [41]:
## Aggregate the data by county
# Group by 'County' and calculate the mean of the 'Baseline.Health' column
county_health_data = CVI_pct_comb_baseline.groupby(['State','County'])['Baseline.Health'].mean()
county_health_data

State  County        
AK                       0.556897
       Aleutians East    0.339310
       Aleutians West    0.402766
       Anchorage         0.393992
       Bethel            0.715568
                           ...   
WY     Sweetwater        0.416394
       Teton             0.193169
       Uinta             0.402018
       Washakie          0.434186
       Weston            0.416094
Name: Baseline.Health, Length: 3136, dtype: float64

In [42]:
# Show the county ranked from lowest to highest 'Baseline.Health' score
county_health_data.loc['MA'].sort_values()

County
Nantucket     0.232253
Middlesex     0.281394
Norfolk       0.299251
Dukes         0.366173
Hampshire     0.376519
Plymouth      0.411366
Suffolk       0.428372
Essex         0.429805
Barnstable    0.430583
Franklin      0.435194
Worcester     0.440516
Berkshire     0.499086
Bristol       0.523408
Hampden       0.591569
Name: Baseline.Health, dtype: float64

Comparing the values above with the data from https://www.countyhealthrankings.org/health-data/massachusetts?year=2024&measure=Health+Outcomes

1. Health Outcomes tell us how long people live on average within a community, and how much physical and mental health people experience in a community while they are alive.
(It seems that this score is in goog agreement with the Toxpi index found before using the baseline vulnerabilities)

<img src="2024-health-outcomes-massachusetts.png" alt="Image of Massachussetts shown Health Outcomes by county" width="900"/>

2. Many things influence how well and how long we live. Health Factors represent those things we can improve to live longer and healthier lives. They are indicators of the future health of our communities.

<img src="2024-health-factors-massachusetts.png" alt="Image of Massachussetts shown Health Factors by county" width="900"/>



In [43]:
# Renaming the columns so that they match the validation data column names
health_county_validation_data = health_county_validation_data.rename(columns={
    'Unnamed: 1': 'State',
    'Unnamed: 2': 'County',
    'Health Outcomes': 'National-Z-Score-health-outcomes'
})

In [44]:
health_county_validation_data.groupby('County')['National-Z-Score-health-outcomes']
health_county_validation_data = health_county_validation_data[['State', 'County', 'National-Z-Score-health-outcomes']]
# Rewriting the State columns in the two leter format for states. 
health_county_validation_data['State'] = health_county_validation_data['State'].map(state_abbreviations)
# Droping NaN
health_county_validation_data.dropna(inplace=True)
health_county_validation_data.head()

,State,County,National-Z-Score-health-outcomes
2,AL,Autauga,0.133257
3,AL,Baldwin,-0.223533
4,AL,Barbour,1.346811
5,AL,Bibb,0.825542
6,AL,Blount,0.300239


In [45]:
## Aggregate the data by county CVI data
# Group by 'County' and calculate the mean of the 'Baseline.' column
county_health_data = CVI_pct_comb_baseline.groupby(['State','County'])['Baseline.Health'].mean()
county_health_data.reset_index()

,State,County,Baseline.Health
0,AK,,0.556897
1,AK,Aleutians East,0.339310
2,AK,Aleutians West,0.402766
3,AK,Anchorage,0.393992
4,AK,Bethel,0.715568
...,...,...,...
3131,WY,Sweetwater,0.416394
3132,WY,Teton,0.193169
3133,WY,Uinta,0.402018
3134,WY,Washakie,0.434186


In [46]:
# Merge the two previous data frames on the 'County' and 'State' column to then get the Spearman index
merged_data_health = pd.merge(health_county_validation_data, county_health_data, on=['State','County'])
merged_data_health

,State,County,National-Z-Score-health-outcomes,Baseline.Health
0,AL,Autauga,0.133257,0.618420
1,AL,Baldwin,-0.223533,0.550053
2,AL,Barbour,1.346811,0.812175
3,AL,Bibb,0.825542,0.705621
4,AL,Blount,0.300239,0.697988
...,...,...,...,...
3038,WY,Sweetwater,0.014509,0.416394
3039,WY,Teton,-1.448624,0.193169
3040,WY,Uinta,0.076762,0.402018
3041,WY,Washakie,-0.727387,0.434186


In [47]:
# Calculate Spearman correlation for each state
spearman_results = {}
for state_name, abbrev in state_abbreviations.items():
    state_data = merged_data_health[merged_data_health['State'] == abbrev]
    if not state_data.empty:
        correlation, p_value = spearmanr(state_data['National-Z-Score-health-outcomes'], state_data['Baseline.Health'])
        spearman_results[abbrev] = {'Spearman Correlation': correlation, 'p-value': p_value}

# Convert the results dictionary to a DataFrame for better visualization
spearman_results_df = pd.DataFrame.from_dict(spearman_results, orient='index')
spearman_results_df

,Spearman Correlation,p-value
AL,0.764586,5.085410e-14
AK,0.768492,2.945927e-05
AZ,0.696429,3.921191e-03
AR,0.783642,9.499039e-17
CA,0.858747,6.722528e-18
CO,0.876622,9.302973e-20
CT,0.833333,1.017554e-02
DE,1.000000,0.000000e+00
FL,0.904382,9.899170e-26
GA,0.809413,3.960262e-38


In [48]:
# Sorting by Spearman correlation 
spearman_results_df.sort_values('Spearman Correlation',ascending=False)

,Spearman Correlation,p-value
DE,1.000000,0.000000e+00
RI,1.000000,1.404265e-24
NJ,0.954545,1.936325e-11
ME,0.950000,1.834617e-08
MD,0.933794,7.751322e-11
MA,0.916484,4.084159e-06
NC,0.905887,2.403704e-38
FL,0.904382,9.899170e-26
VA,0.897956,6.472118e-35
OH,0.896129,4.339308e-32


In [49]:
# Sorting by P-value 
spearman_results_df.sort_values('p-value',ascending=False)

,Spearman Correlation,p-value
HI,0.800000,2.000000e-01
NV,0.400000,1.247504e-01
CT,0.833333,1.017554e-02
VT,0.665934,9.322999e-03
AZ,0.696429,3.921191e-03
NH,0.878788,8.138621e-04
ID,0.530353,2.540864e-04
WY,0.726285,8.714738e-05
AK,0.768492,2.945927e-05
UT,0.725780,1.237508e-05


### Validation of of the Baseline Environment (done)

In [50]:
## Aggregate the data by county
# Group by 'County' and calculate the mean of the 'Baseline.Health' column
county_Environment_data = CVI_pct_comb_baseline.groupby(['State','County'])['Baseline.Environment'].mean()
#county_Environment_data = CVI_pct_comb_baseline.groupby(['State','County'])['Pollution Sources (Baseline.Environment)'].mean()
county_Environment_data

State  County        
AK                       0.194840
       Aleutians East    0.187074
       Aleutians West    0.228353
       Anchorage         0.450055
       Bethel            0.204944
                           ...   
WY     Sweetwater        0.340435
       Teton             0.180752
       Uinta             0.308237
       Washakie          0.222586
       Weston            0.338110
Name: Baseline.Environment, Length: 3136, dtype: float64

In [51]:
# Show the county ranked from lowest to highest 'Baseline.Infrastructure_' score
county_Environment_data.loc['MA'].sort_values()

County
Dukes         0.107577
Nantucket     0.119300
Barnstable    0.248096
Berkshire     0.266908
Franklin      0.298862
Hampshire     0.334675
Plymouth      0.357546
Worcester     0.408311
Bristol       0.438415
Norfolk       0.471698
Essex         0.477177
Middlesex     0.497713
Hampden       0.506758
Suffolk       0.575069
Name: Baseline.Environment, dtype: float64

Comparing the values above with the data from https://www.countyhealthrankings.org/health-data/massachusetts?year=2024&measure=Health+Outcomes

1. Physical Enviroment air polution
(It seems that this score is in goog agreement with the Toxpi index found before using the baseline vulnerabilities)

<img src="Infrastructure.png" alt="Image of Massachussetts shown by environment Outcomes by county" width="900"/>

In [52]:
# Environment data from the validation data

infra_county_validation_data

,State,County,National Z-Score
0,Arkansas,Arkansas,NaN
1,Arkansas,Arkansas,0.19
2,Arkansas,Ashley,0.37
3,Arkansas,Baxter,0.25
4,Arkansas,Benton,1.08
...,...,...,...
2820,Colorado,Summit,-2.24
2821,Colorado,Teller,-1.89
2822,Colorado,Washington,-1.41
2823,Colorado,Weld,0.84


In [53]:
# Resetting the index
county_Environment_data.reset_index()


,State,County,Baseline.Environment
0,AK,,0.194840
1,AK,Aleutians East,0.187074
2,AK,Aleutians West,0.228353
3,AK,Anchorage,0.450055
4,AK,Bethel,0.204944
...,...,...,...
3131,WY,Sweetwater,0.340435
3132,WY,Teton,0.180752
3133,WY,Uinta,0.308237
3134,WY,Washakie,0.222586


In [54]:
# Rewriting the State columns in the two leter format for states. 
infra_county_validation_data['State'] = infra_county_validation_data['State'].map(state_abbreviations)
# Droping NaN
infra_county_validation_data.dropna(inplace=True)
infra_county_validation_data.head()

,State,County,National Z-Score
1,AR,Arkansas,0.19
2,AR,Ashley,0.37
3,AR,Baxter,0.25
4,AR,Benton,1.08
5,AR,Boone,0.19


In [55]:
# Merge the two previous data frames on the 'County' and 'State' column to then get the Spearman index
merged_data_Environment = pd.merge(infra_county_validation_data, county_Environment_data, on=['State','County'])
merged_data_Environment

,State,County,National Z-Score,Baseline.Environment
0,AR,Arkansas,0.19,0.331966
1,AR,Ashley,0.37,0.325237
2,AR,Baxter,0.25,0.331845
3,AR,Benton,1.08,0.367384
4,AR,Boone,0.19,0.295767
...,...,...,...,...
2592,CO,Summit,-2.24,0.215513
2593,CO,Teller,-1.89,0.211525
2594,CO,Washington,-1.41,0.386793
2595,CO,Weld,0.84,0.546176


In [56]:
# Calculate Spearman correlation for each state
spearman_results = {}
for state_name, abbrev in state_abbreviations.items():
    state_data = merged_data_Environment[merged_data_Environment['State'] == abbrev]
    if not state_data.empty:
        correlation, p_value = spearmanr(state_data['National Z-Score'], state_data['Baseline.Environment'])
        spearman_results[abbrev] = {'Spearman Correlation': correlation, 'p-value': p_value}

# Convert the results dictionary to a DataFrame for better visualization
spearman_results_env_df = pd.DataFrame.from_dict(spearman_results, orient='index')
spearman_results_env_df

,Spearman Correlation,p-value
AL,0.176806,1.523488e-01
AZ,0.306178,2.670538e-01
AR,0.340218,2.821662e-03
CA,0.742038,2.646960e-11
CO,0.824764,9.773058e-16
CT,0.831386,1.052013e-02
DE,0.866025,3.333333e-01
FL,0.034429,7.820951e-01
ID,0.065574,6.761113e-01
IN,0.031999,7.620399e-01


In [57]:
# Sorting by Spearman correlation
spearman_results_env_df.sort_values('Spearman Correlation',ascending=False)

,Spearman Correlation,p-value
DE,0.866025,3.333333e-01
CT,0.831386,1.052013e-02
CO,0.824764,9.773058e-16
MI,0.793534,3.718492e-19
CA,0.742038,2.646960e-11
NC,0.729381,7.598625e-18
MA,0.682821,7.117940e-03
WI,0.659535,2.972695e-10
NJ,0.649545,1.439421e-03
UT,0.638071,2.591123e-04


In [58]:
# Sorting by P-value
spearman_results_env_df.sort_values('p-value',ascending=False)

,Spearman Correlation,p-value
NE,0.000950,9.933698e-01
KS,0.007587,9.390741e-01
MT,-0.023820,8.723282e-01
NH,0.086164,8.129093e-01
FL,0.034429,7.820951e-01
IN,0.031999,7.620399e-01
ID,0.065574,6.761113e-01
SC,0.105637,4.847347e-01
TX,0.046540,4.692899e-01
RI,0.461690,4.337662e-01


### Validation of the Socio-economic baseline (done)

In [59]:
## Aggregate the data by county
# Group by 'County' and calculate the mean of the 'Baseline.' column
county_SocialEconomic_data = CVI_pct_comb_baseline.groupby(['State','County'])['Baseline.SocialEconomic'].mean()
county_SocialEconomic_data

State  County        
AK                       0.501905
       Aleutians East    0.527355
       Aleutians West    0.526443
       Anchorage         0.542489
       Bethel            0.742275
                           ...   
WY     Sweetwater        0.419149
       Teton             0.268879
       Uinta             0.451767
       Washakie          0.481516
       Weston            0.508545
Name: Baseline.SocialEconomic, Length: 3136, dtype: float64

In [60]:
# Show the county ranked from lowest to highest 'Baseline.Health' score
county_SocialEconomic_data.loc['MA'].sort_values()

County
Nantucket     0.316830
Dukes         0.362638
Hampshire     0.386988
Middlesex     0.389582
Norfolk       0.407265
Barnstable    0.431411
Plymouth      0.467932
Essex         0.485680
Worcester     0.491221
Berkshire     0.519233
Bristol       0.520377
Franklin      0.539798
Suffolk       0.584281
Hampden       0.637128
Name: Baseline.SocialEconomic, dtype: float64

Comparing the values above with the data from https://eig.org/2022-dci-community-rankings/

1. Social and Economic data comparison with the more prosperous and distressed counties in the US using the Distressed Communities Index

<img src="Distressed-comm-index.png" alt="Image of Massachussetts shown by by county" width="900"/>

In [61]:
# Rename columns in population_data to match those used in other data processing steps
population_data.rename(columns={'Stabr': 'State', 'Area_name': 'County'}, inplace=True)


# Rename columns in soecon_county_validation_data for consistency
soecon_county_validation_data.rename(columns={'Stabr': 'State', 'Area_name': 'County'}, inplace=True)

In [62]:
# Aggregate and process data. One state only
county_SocialEconomic_data = CVI_pct_comb_baseline.groupby(['State', 'County'])['Baseline.SocialEconomic'].mean()
state = 'MA'
cvi_data = county_SocialEconomic_data.loc[state].reset_index()
cvi_data.columns = ['County', 'Baseline.SocialEconomic']
# Ensure 'County' is treated as a column in cvi_data
cvi_data.reset_index(drop=True, inplace=True)

Npoor_df = soecon_county_validation_data.groupby(['State', 'County'])['POVALL_2021'].mean().loc[state]
Npopu_df = population_data.groupby(['State', 'County'])['POPESTIMATE2021'].mean().loc[state]

# Convert Npoor_df to numeric, coercing any errors which will turn non-numeric values to NaN
Npoor_df = pd.to_numeric(Npoor_df, errors='coerce')
Npopu_df = pd.to_numeric(Npopu_df, errors='coerce')

# Create a DataFrame for percentage of poor
percentage_poor = pd.DataFrame({
    'County': Npoor_df.index,
    'Percentage_Poor': (Npoor_df / Npopu_df * 100)
}).reset_index(drop=True)
percentage_poor['Percentage_Poor'] = pd.to_numeric(percentage_poor['Percentage_Poor'], errors='coerce')
percentage_poor.dropna(inplace=True)
percentage_poor.sort_values(by='Percentage_Poor', inplace=True)

# Reset the index for both dataframes if 'County' is part of the index
percentage_poor.reset_index(drop=True, inplace=True)
cvi_data.reset_index(drop=True, inplace=True)
# Remove the row where 'County' is 'Massachusetts'
# Create inverse dictionary for state abbreviations
inverse_state_abbreviations = {v: k for k, v in state_abbreviations.items()}
# Remove the row where 'County' is State Name (e.g. 'Massachusetts')
percentage_poor = percentage_poor[percentage_poor['County'] != inverse_state_abbreviations.get(state)] 

# Strip whitespace and convert to lowercase for consistency in both DataFrames
percentage_poor['County'] = percentage_poor['County'].str.strip().str.lower()
cvi_data['County'] = cvi_data['County'].str.strip().str.lower()


# Merge the datasets on the 'County' column
merged_data = pd.merge(percentage_poor, cvi_data, on='County')

# Calculate the Spearman correlation
if not merged_data.empty:
    spearman_corr, _ = spearmanr(merged_data['Percentage_Poor'], merged_data['Baseline.SocialEconomic'])
    print(f"Spearman's Rank Correlation Coefficient: {spearman_corr}")
else:
    print("No data to compare. Check the county names and state filters.")


Spearman's Rank Correlation Coefficient: 0.832967032967033


In [63]:
# Initialize a DataFrame to store results
results = pd.DataFrame(columns=['State', 'Spearman_Correlation'])

for state_name, state_abbr in state_abbreviations.items():
    print(f"Processing data for {state_name} ({state_abbr})")

    # Extract and align poverty and population data
    Npoor_df = soecon_county_validation_data.groupby(['State', 'County'])['POVALL_2021'].mean().loc[state_abbr]
    Npopu_df = population_data.groupby(['State', 'County'])['POPESTIMATE2021'].mean().loc[state_abbr]
    aligned_poor, aligned_pop = Npoor_df.align(Npopu_df)

    # Convert Npoor_df to numeric, coercing any errors which will turn non-numeric values to NaN
    Npoor_df = pd.to_numeric(Npoor_df, errors='coerce')
    Npopu_df = pd.to_numeric(Npopu_df, errors='coerce')

    # Calculate the percentage of poor population
    percentage_poor = pd.DataFrame({
        'County': aligned_poor.index,
        'Percentage_Poor': (aligned_poor / aligned_pop * 100)
    }).reset_index(drop=True)
    percentage_poor['Percentage_Poor'] = pd.to_numeric(percentage_poor['Percentage_Poor'], errors='coerce')
    percentage_poor.dropna(inplace=True)
    percentage_poor.sort_values(by='Percentage_Poor', inplace=True)
    percentage_poor.reset_index(drop=True, inplace=True)
    
    # Process social economic baseline data
    cvi_data = CVI_pct_comb_baseline.groupby(['State', 'County'])['Baseline.SocialEconomic'].mean().loc[state_abbr].reset_index()
    cvi_data.columns = ['County', 'Baseline.SocialEconomic']
    cvi_data.reset_index(drop=True, inplace=True)
    percentage_poor['County'] = percentage_poor['County'].str.strip().str.lower()
    cvi_data['County'] = cvi_data['County'].str.strip().str.lower()

    # Merge datasets on 'County'
    merged_data = pd.merge(percentage_poor, cvi_data, on='County')

    # Calculate the Spearman correlation
    if not merged_data.empty:
        spearman_corr, p_value = spearmanr(merged_data['Percentage_Poor'], merged_data['Baseline.SocialEconomic'])
        new_row = pd.DataFrame({'State': [state_name], 'Spearman_Correlation': [spearman_corr],'P_Value': [p_value]})
        results = pd.concat([results, new_row], ignore_index=True)
    else:
        new_row = pd.DataFrame({'State': [state_name], 'Spearman_Correlation': [pd.NA]})
        results = pd.concat([results, new_row], ignore_index=True)


Processing data for Alabama (AL)
Processing data for Alaska (AK)
Processing data for Arizona (AZ)
Processing data for Arkansas (AR)
Processing data for California (CA)
Processing data for Colorado (CO)
Processing data for Connecticut (CT)
Processing data for Delaware (DE)
Processing data for Florida (FL)
Processing data for Georgia (GA)
Processing data for Hawaii (HI)
Processing data for Idaho (ID)


Processing data for Illinois (IL)
Processing data for Indiana (IN)
Processing data for Iowa (IA)
Processing data for Kansas (KS)
Processing data for Kentucky (KY)
Processing data for Louisiana (LA)
Processing data for Maine (ME)
Processing data for Maryland (MD)
Processing data for Massachusetts (MA)
Processing data for Michigan (MI)
Processing data for Minnesota (MN)
Processing data for Mississippi (MS)
Processing data for Missouri (MO)
Processing data for Montana (MT)
Processing data for Nebraska (NE)
Processing data for Nevada (NV)
Processing data for New Hampshire (NH)
Processing data for New Jersey (NJ)
Processing data for New Mexico (NM)
Processing data for New York (NY)
Processing data for North Carolina (NC)
Processing data for North Dakota (ND)
Processing data for Ohio (OH)
Processing data for Oklahoma (OK)
Processing data for Oregon (OR)
Processing data for Pennsylvania (PA)
Processing data for Rhode Island (RI)
Processing data for South Carolina (SC)
Processing data for Sout

In [64]:
# Sort 'results' DataFrame based on 'Spearman_Correlation' column in descending order
sorted_results = results.sort_values('Spearman_Correlation', ascending=False)

# Display the sorted DataFrame
sorted_results

,State,Spearman_Correlation,P_Value
18,Maine,0.905882,1.363513e-06
19,Maryland,0.901186,4.486811e-09
29,New Jersey,0.871429,2.708908e-07
28,New Hampshire,0.842424,2.220031e-03
20,Massachusetts,0.832967,2.165509e-04
5,Colorado,0.808883,6.203090e-16
39,South Carolina,0.776873,2.197705e-10
34,Ohio,0.769364,2.022992e-18
4,California,0.766588,2.309329e-12
32,North Carolina,0.745911,5.401995e-19


In [65]:
# Calculate the mean of the Spearman correlation at the National level
sorted_results['Spearman_Correlation'].mean()

0.6106225016324517

In [66]:
# Sort 'results' DataFrame based on 'Spearman_Correlation' column in descending order
sorted_results_pvalue = results.sort_values('P_Value', ascending=False)

# Display the sorted DataFrame
sorted_results_pvalue

,State,Spearman_Correlation,P_Value
7,Delaware,0.500000,6.666667e-01
10,Hawaii,0.359092,5.528147e-01
26,Nebraska,0.121199,2.471641e-01
38,Rhode Island,0.700000,1.881204e-01
33,North Dakota,0.250685,7.021361e-02
46,Washington,0.321660,4.584366e-02
49,Wyoming,0.472332,2.285864e-02
2,Arizona,0.610714,1.559283e-02
44,Vermont,0.661538,9.975179e-03
11,Idaho,0.441582,2.392371e-03


P values above 10^-2 are not statistically significant. Therefore Delaware, Hawaii, Nebraska and 
Rhode Island have changes more than 18% to be true under the null hypothesis of no correlation. For the other states we get a high significance. In my opinion the this is sufficient to show the CVI is a strong index. 

### Validation of the Infrastructure baseline (done)

In [67]:
## Aggregate the data by county
# Group by 'County' and calculate the mean of the 'Baseline.Infrastructure' column
county_Infrastructure_data = CVI_pct_comb_baseline.groupby(['State','County'])['Baseline.Infrastructure'].mean()
#county_Infrastructure_data = CVI_pct_comb_baseline.groupby(['State','County'])['Communications (Baseline.Infrastructure)'].mean()
#county_Environment_data = CVI_pct_comb_baseline.groupby(['State','County'])['Pollution Sources (Baseline.Environment)'].mean()
county_Infrastructure_data

State  County        
AK                       0.624348
       Aleutians East    0.636090
       Aleutians West    0.517467
       Anchorage         0.448055
       Bethel            0.688104
                           ...   
WY     Sweetwater        0.462261
       Teton             0.345712
       Uinta             0.437879
       Washakie          0.418665
       Weston            0.429222
Name: Baseline.Infrastructure, Length: 3136, dtype: float64

In [68]:
# Resetting the index
county_Infrastructure_data.reset_index()

,State,County,Baseline.Infrastructure
0,AK,,0.624348
1,AK,Aleutians East,0.636090
2,AK,Aleutians West,0.517467
3,AK,Anchorage,0.448055
4,AK,Bethel,0.688104
...,...,...,...
3131,WY,Sweetwater,0.462261
3132,WY,Teton,0.345712
3133,WY,Uinta,0.437879
3134,WY,Washakie,0.418665


In [69]:
# Rewriting the State columns in the two leter format for states. 
infrastructure_county_validation_data['State'] = infrastructure_county_validation_data['State'].map(state_abbreviations)
# Droping NaN
infrastructure_county_validation_data.dropna(inplace=True)
infrastructure_county_validation_data.head()

,State,County,County Value**,# Households with Broadband Access,Error Margin
0,CO,Colorado,92%,"2,091,383",92-92%
1,CO,Adams,91%,"165,088",90-92
2,CO,Alamosa,89%,"5,757",86-92
3,CO,Arapahoe,94%,"234,735",93-94
4,CO,Archuleta,90%,"5,136",86-93


In [70]:
# Merge the two previous data frames on the 'County' and 'State' column to then get the Spearman index
merged_data_Infrastructure = pd.merge(infrastructure_county_validation_data, county_Infrastructure_data, on=['State','County'])
merged_data_Infrastructure

,State,County,County Value**,# Households with Broadband Access,Error Margin,Baseline.Infrastructure
0,CO,Adams,91%,"165,088",90-92,0.376907
1,CO,Alamosa,89%,"5,757",86-92,0.515342
2,CO,Arapahoe,94%,"234,735",93-94,0.239368
3,CO,Archuleta,90%,"5,136",86-93,0.505056
4,CO,Baca,81%,"1,280",74-87,0.553321
...,...,...,...,...,...,...
3008,OH,Washington,83%,"20,432",81-85,0.516278
3009,OH,Wayne,83%,"36,545",81-84,0.441940
3010,OH,Williams,85%,"13,108",83-88,0.529642
3011,OH,Wood,91%,"48,680",89-92,0.449443


In [71]:
# Calculate Spearman correlation for each state
# Calculate Spearman correlation for each state
spearman_results = {}
for state_name, abbrev in state_abbreviations.items():
    state_data = merged_data_Infrastructure[merged_data_Infrastructure['State'] == abbrev]
    if not state_data.empty:
        correlation, p_value = spearmanr(state_data['County Value**'], state_data['Baseline.Infrastructure'])
        spearman_results[abbrev] = {'Spearman Correlation': correlation, 'p-value': p_value}

# Convert the results dictionary to a DataFrame for better visualization
spearman_results_inf_df = pd.DataFrame.from_dict(spearman_results, orient='index')
spearman_results_inf_df

,Spearman Correlation,p-value
AL,-0.489412,2.635076e-05
AZ,-0.477063,7.215393e-02
AR,-0.589889,2.055302e-08
CA,-0.750590,1.168494e-11
CO,-0.744359,1.795372e-12
CT,-0.600751,1.152637e-01
DE,-0.500000,6.666667e-01
FL,-0.744598,5.090152e-13
GA,-0.650799,1.609382e-20
HI,-0.264706,6.122150e-01


In [72]:
# Sorting by Spearman correlation. Values should be negative because the higher % of houses with internet,  the better the infrastructure and lower Baseline.Infrastructure value
spearman_results_inf_df.sort_values('Spearman Correlation',ascending=True)

,Spearman Correlation,p-value
NJ,-0.882445,1.209483e-07
ME,-0.808584,1.496790e-04
SC,-0.774209,2.769703e-10
NY,-0.767551,2.165483e-13
CA,-0.750590,1.168494e-11
WI,-0.745360,5.945111e-14
MD,-0.744981,4.541947e-05
FL,-0.744598,5.090152e-13
CO,-0.744359,1.795372e-12
VA,-0.734865,2.309919e-17


In [73]:
# Sorting by P-value
spearman_results_inf_df.sort_values('p-value',ascending=False)

,Spearman Correlation,p-value
RI,-0.051299,9.347128e-01
DE,-0.500000,6.666667e-01
HI,-0.264706,6.122150e-01
VT,-0.392038,1.656345e-01
NM,-0.256561,1.563544e-01
NH,-0.489317,1.511830e-01
CT,-0.600751,1.152637e-01
NV,-0.420477,9.284634e-02
WY,-0.377621,7.565731e-02
AZ,-0.477063,7.215393e-02
